In [2]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error


In [3]:
!pip install pyarrow

# Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [4]:
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [5]:
len(df_jan.columns)
# Q1 Answer: 19

19

In [6]:
df_jan.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [7]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


# Q2. Computing duration
Now let's compute the 'duration' variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [8]:
def read_dataframe(df):
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds() / 60)

    categorical = ['PULocationID', 'DOLocationID']
    df.loc[:, categorical] = df.loc[:, categorical].astype(str)
    numerical = ['trip_distance']

    
    return df

df_jan = read_dataframe(df_jan)
df_feb = read_dataframe(df_feb)

print(np.std(df_jan.duration))
# Q2 Answer: 42.59

42.5943442974141


# Q3. Dropping outliers
Next, we need to check the distribution of the 'duration' variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [9]:
total_records = len(df_jan)

df_jan = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]
df_feb = df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)]

remaining_records = len(df_jan)

fraction_left = remaining_records / total_records

print(fraction_left)
# Q3 Answer: 98%

0.9812202822125979


# Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)

* Fit a dictionary vectorizer
* Get a feature matrix from it
* What's the dimensionality of this matrix (number of columns)?

In [10]:
dv = DictVectorizer()

train_dicts = df_jan[['PULocationID', 'DOLocationID']].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_feb[['PULocationID', 'DOLocationID']].to_dict(orient='records')
X_val = dv.transform(val_dicts)

X_train.shape[1]
# Q4 Answer: 515

515

# Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters.
Calculate the RMSE of the model on the training data.

What's the RMSE on train?

In [11]:
y = df_jan['duration']

model = LinearRegression()
model.fit(X_train, y)

y_pred = model.predict(X_train)

rmse = mean_squared_error(y, y_pred, squared=False)
rmse
# Q5 Answer: 7.64

7.6492610279057605

# Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [14]:

y_val = df_feb['duration']

y_val_pred = model.predict(X_val)
rmse_val = mean_squared_error(y_val, y_val_pred, squared=False)
rmse_val
# Q6 Answer: 7.81

7.81183265470218